# 02 - Data Preprocessing & Feature Engineering

This notebook handles data preprocessing including:
- Missing value handling
- Skills multi-hot encoding with frequency filtering
- Categorical encoding (ordinal and one-hot)
- Feature scaling
- Train/test split

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

## 1. Load Dataset

In [2]:
df = pd.read_csv('../data/ai_job_dataset.csv')
print(f"Original dataset shape: {df.shape}")
df.head()

Original dataset shape: (15000, 19)


,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,AI00001,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,AI00002,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,AI00003,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,AI00004,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,AI00005,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics


## 2. Select Features & Target

In [3]:
# Columns to drop (identifiers, leakage-prone, low predictive value)
drop_cols = ['job_id', 'salary_currency', 'employee_residence', 'posting_date', 
             'application_deadline', 'job_description_length', 'company_name']

# Target variable
target = 'salary_usd'

# Create working dataframe
df_work = df.drop(columns=drop_cols)
print(f"Working dataset shape: {df_work.shape}")
print(f"\nColumns: {df_work.columns.tolist()}")

Working dataset shape: (15000, 12)

Columns: ['job_title', 'salary_usd', 'experience_level', 'employment_type', 'company_location', 'company_size', 'remote_ratio', 'required_skills', 'education_required', 'years_experience', 'industry', 'benefits_score']


## 3. Missing Value Handling

In [4]:
# Check missing values
missing = df_work.isnull().sum()
missing_pct = (missing / len(df_work)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
print("Missing values summary:")
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False))

if missing.sum() == 0:
    print("\nNo missing values found!")

Missing values summary:
Empty DataFrame
Columns: [Missing Count, Missing %]
Index: []

No missing values found!


## 3.5 Outlier Handling (IQR-Based Capping)

Cap extreme salary values using the IQR method (winsorization) to reduce the impact of outliers without dropping rows.

In [ ]:
# IQR-based winsorization on salary_usd
Q1 = df_work['salary_usd'].quantile(0.25)
Q3 = df_work['salary_usd'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_below = (df_work['salary_usd'] < lower_bound).sum()
outliers_above = (df_work['salary_usd'] > upper_bound).sum()
print(f"Salary IQR bounds: [{lower_bound:,.0f}, {upper_bound:,.0f}]")
print(f"Outliers below: {outliers_below}, above: {outliers_above}")

# Cap (winsorize) instead of dropping
df_work['salary_usd'] = df_work['salary_usd'].clip(lower=lower_bound, upper=upper_bound)
print(f"\nAfter capping:")
print(df_work['salary_usd'].describe())

## 3.6 Salary Log-Transform

Apply `log1p` to normalize the right-skewed salary distribution. Models will be trained on log-scale; predictions will be inverse-transformed back to dollars for evaluation.

In [ ]:
# Apply log1p transform to salary
df_work['salary_log'] = np.log1p(df_work['salary_usd'])

print("Salary distribution before and after log transform:")
print(f"  salary_usd  — skew: {df_work['salary_usd'].skew():.3f}, mean: ${df_work['salary_usd'].mean():,.0f}")
print(f"  salary_log  — skew: {df_work['salary_log'].skew():.3f}, mean: {df_work['salary_log'].mean():.4f}")

# Keep original salary for later inverse-transform evaluation
df_work['salary_original'] = df_work['salary_usd']

## 4. Skills Multi-Hot Encoding

In [5]:
# Extract all unique skills and their frequencies
def extract_skills(df):
    all_skills = []
    for skills_str in df['required_skills']:
        skills = [s.strip() for s in skills_str.split(',')]
        all_skills.extend(skills)
    return pd.Series(all_skills).value_counts()

skill_counts = extract_skills(df_work)
print(f"Total unique skills: {len(skill_counts)}")
print(f"\nSkill frequency distribution:")
print(skill_counts.describe())

Total unique skills: 24

Skill frequency distribution:
count      24.000000
mean     2495.541667
std       572.221603
min      1833.000000
25%      2152.500000
50%      2326.000000
75%      2723.000000
max      4450.000000
Name: count, dtype: float64


In [6]:
# Filter skills that appear in at least 2% of samples
min_freq = 0.02 * len(df_work)
frequent_skills = skill_counts[skill_counts >= min_freq].index.tolist()
print(f"Skills appearing in >= 2% of samples: {len(frequent_skills)}")
print(f"Frequent skills: {frequent_skills}")

Skills appearing in >= 2% of samples: 24
Frequent skills: ['Python', 'SQL', 'TensorFlow', 'Kubernetes', 'Scala', 'PyTorch', 'Linux', 'Git', 'Java', 'GCP', 'Hadoop', 'Tableau', 'R', 'Computer Vision', 'Data Visualization', 'Deep Learning', 'MLOps', 'Spark', 'NLP', 'Azure', 'AWS', 'Mathematics', 'Docker', 'Statistics']


In [7]:
def encode_skills(df, frequent_skills):
    """
    Multi-hot encode skills.
    Returns a DataFrame with binary columns for each frequent skill,
    plus a count of rare skills.
    """
    skill_matrix = pd.DataFrame(index=df.index)
    
    for skill in frequent_skills:
        skill_col = f"skill_{skill.replace(' ', '_').replace('-', '_').lower()}"
        skill_matrix[skill_col] = df['required_skills'].apply(
            lambda x: 1 if skill in [s.strip() for s in x.split(',')] else 0
        )
    
    # Count rare skills (those not in frequent_skills)
    def count_rare_skills(skills_str):
        skills = [s.strip() for s in skills_str.split(',')]
        rare_count = sum(1 for s in skills if s not in frequent_skills)
        return rare_count
    
    skill_matrix['other_skills_count'] = df['required_skills'].apply(count_rare_skills)
    
    return skill_matrix

# Encode skills
skills_encoded = encode_skills(df_work, frequent_skills)
print(f"Skills encoded shape: {skills_encoded.shape}")
skills_encoded.head()

Skills encoded shape: (15000, 25)


,skill_python,skill_sql,skill_tensorflow,skill_kubernetes,skill_scala,skill_pytorch,skill_linux,skill_git,skill_java,skill_gcp,skill_hadoop,skill_tableau,skill_r,skill_computer_vision,skill_data_visualization,skill_deep_learning,skill_mlops,skill_spark,skill_nlp,skill_azure,skill_aws,skill_mathematics,skill_docker,skill_statistics,other_skills_count
0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0
2,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0
3,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


## 5. Ordinal Encoding

In [8]:
# Define ordinal mappings
experience_order = ['EN', 'MI', 'SE', 'EX']  # Entry -> Executive
education_order = ['Associate', 'Bachelor', 'Master', 'PhD']  # Low -> High

# Apply ordinal encoding
df_work['experience_level_enc'] = df_work['experience_level'].map(
    {v: i for i, v in enumerate(experience_order)}
)
df_work['education_required_enc'] = df_work['education_required'].map(
    {v: i for i, v in enumerate(education_order)}
)

print("Experience level encoding:")
print(df_work[['experience_level', 'experience_level_enc']].drop_duplicates().sort_values('experience_level_enc'))
print("\nEducation encoding:")
print(df_work[['education_required', 'education_required_enc']].drop_duplicates().sort_values('education_required_enc'))

Experience level encoding:
   experience_level  experience_level_enc
1                EN                     0
2                MI                     1
0                SE                     2
11               EX                     3

Education encoding:
  education_required  education_required_enc
2          Associate                       0
0           Bachelor                       1
1             Master                       2
3                PhD                       3


## 6. One-Hot Encoding for Nominal Categoricals

In [9]:
# Columns to one-hot encode
onehot_cols = ['job_title', 'company_location', 'industry', 'employment_type', 'company_size']

# Convert remote_ratio to categorical (it's essentially categorical)
df_work['remote_ratio'] = df_work['remote_ratio'].astype(str)
onehot_cols.append('remote_ratio')

# One-hot encode
df_onehot = pd.get_dummies(df_work[onehot_cols], prefix=onehot_cols, drop_first=False)
print(f"One-hot encoded shape: {df_onehot.shape}")
print(f"One-hot columns: {df_onehot.columns.tolist()[:10]}...")

One-hot encoded shape: (15000, 65)
One-hot columns: ['job_title_AI Architect', 'job_title_AI Consultant', 'job_title_AI Product Manager', 'job_title_AI Research Scientist', 'job_title_AI Software Engineer', 'job_title_AI Specialist', 'job_title_Autonomous Systems Engineer', 'job_title_Computer Vision Engineer', 'job_title_Data Analyst', 'job_title_Data Engineer']...


## 7. Combine All Features

In [ ]:
# Numerical features
numerical_cols = ['years_experience', 'benefits_score']

# Ordinal encoded features
ordinal_cols = ['experience_level_enc', 'education_required_enc']

# Combine all features
X = pd.concat([
    df_work[numerical_cols],
    df_work[ordinal_cols],
    df_onehot,
    skills_encoded
], axis=1)

# Use log-transformed salary as target for training
y = df_work['salary_log']

# Keep original-scale salary for evaluation
y_original = df_work['salary_original']

print(f"Final feature matrix shape: {X.shape}")
print(f"Target shape (log-scale): {y.shape}")
print(f"Target shape (original $): {y_original.shape}")

In [11]:
# Display feature summary
print("Feature categories:")
print(f"  - Numerical: {len(numerical_cols)} features")
print(f"  - Ordinal encoded: {len(ordinal_cols)} features")
print(f"  - One-hot encoded: {df_onehot.shape[1]} features")
print(f"  - Skills encoded: {skills_encoded.shape[1]} features")
print(f"  - Total: {X.shape[1]} features")

Feature categories:
  - Numerical: 2 features
  - Ordinal encoded: 2 features
  - One-hot encoded: 65 features
  - Skills encoded: 25 features
  - Total: 94 features


## 8. Train / Validation / Test Split (70/10/20)

In [ ]:
# Two-stage split: first 70/30, then split the 30 into 10 val + 20 test
strat_col = df_work['experience_level']

# Stage 1: 70% train, 30% temp
X_train, X_temp, y_train, y_temp, strat_train, strat_temp = train_test_split(
    X, y, strat_col,
    test_size=0.30,
    random_state=42,
    stratify=strat_col
)

# Also split original-scale targets in parallel
y_train_original = y_original.loc[X_train.index]
y_temp_original = y_original.loc[X_temp.index]

# Stage 2: split 30% into 1/3 val (10% overall) + 2/3 test (20% overall)
X_val, X_test, y_val, y_test, strat_val, strat_test = train_test_split(
    X_temp, y_temp, strat_temp,
    test_size=2/3,
    random_state=42,
    stratify=strat_temp
)

y_val_original = y_temp_original.loc[X_val.index]
y_test_original = y_temp_original.loc[X_test.index]

print(f"Training set:   {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"Validation set: {X_val.shape[0]} samples ({X_val.shape[0]/len(X)*100:.0f}%)")
print(f"Test set:       {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.0f}%)")
print(f"\nTarget (log-scale) distribution in training set:")
print(y_train.describe())

## 9. Feature Scaling

In [ ]:
# Scale numerical features (for linear models and MLP)
# Tree-based models don't need scaling, but we'll create scaled version anyway

scaler = StandardScaler()

# Columns to scale
cols_to_scale = numerical_cols + ['other_skills_count']

# Fit scaler on training data only
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()

X_train_scaled[cols_to_scale] = scaler.fit_transform(X_train[cols_to_scale])
X_val_scaled[cols_to_scale] = scaler.transform(X_val[cols_to_scale])
X_test_scaled[cols_to_scale] = scaler.transform(X_test[cols_to_scale])

print("Scaling applied to:", cols_to_scale)
print(f"\nScaled training data sample statistics for 'years_experience':")
print(f"  Mean: {X_train_scaled['years_experience'].mean():.4f}")
print(f"  Std: {X_train_scaled['years_experience'].std():.4f}")

## 10. Save Preprocessed Data

In [ ]:
import os

# Create output directory
os.makedirs('../data/processed', exist_ok=True)

# Save unscaled data (for tree-based models)
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_val.to_csv('../data/processed/X_val.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)

# Save log-scale targets (for model training)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_val.to_csv('../data/processed/y_val.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

# Save original-scale targets (for evaluation in dollar terms)
y_train_original.to_csv('../data/processed/y_train_original.csv', index=False)
y_val_original.to_csv('../data/processed/y_val_original.csv', index=False)
y_test_original.to_csv('../data/processed/y_test_original.csv', index=False)

# Save scaled data (for linear models and MLP)
X_train_scaled.to_csv('../data/processed/X_train_scaled.csv', index=False)
X_val_scaled.to_csv('../data/processed/X_val_scaled.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test_scaled.csv', index=False)

# Save preprocessing artifacts
preprocessing_artifacts = {
    'scaler': scaler,
    'frequent_skills': frequent_skills,
    'experience_order': experience_order,
    'education_order': education_order,
    'onehot_cols': onehot_cols,
    'feature_columns': X.columns.tolist(),
    'cols_to_scale': cols_to_scale
}

joblib.dump(preprocessing_artifacts, '../data/processed/preprocessing_artifacts.joblib')

print("Preprocessed data saved to ../data/processed/")
print("Files:")
for f in sorted(os.listdir('../data/processed')):
    print(f"  - {f}")

## Summary

### Preprocessing Steps Completed:
1. **Dropped columns**: Removed identifiers and low-value features
2. **Missing values**: Checked and handled (if any)
3. **Outlier handling**: IQR-based winsorization on salary_usd
4. **Log-transform**: Applied `log1p` to salary for normalized target distribution
5. **Skills encoding**: Multi-hot encoded frequent skills (>= 2%), rare skills counted
6. **Ordinal encoding**: experience_level (EN→EX), education_required (Associate→PhD)
7. **One-hot encoding**: job_title, company_location, industry, employment_type, company_size, remote_ratio
8. **Train/Val/Test split**: 70/10/20, stratified by experience_level
9. **Scaling**: StandardScaler on numerical features (fitted on training data only)

### Output:
- Unscaled data for tree-based models (Random Forest, XGBoost, Decision Tree, Gradient Boosting)
- Scaled data for linear models and neural networks
- Original-scale targets for dollar-based evaluation
- Preprocessing artifacts for inference pipeline